In [1]:
import numpy as np
import functools 
import mnist_reader
from scipy.optimize import minimize

In [2]:
def inflate_matrixes(flat_thetas, shapes):
    layers  = len(shapes)+1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers-1):
        steps[i+1] = steps[i] + sizes[i]
    return[
        flat_thetas[steps[i]: steps[i+1]].reshape(*shapes[i])
        for i in range(layers-1)
    ]


In [3]:
def feed_forward(thetas, X): # Es sera de (784, 60000)
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul( 
                    thetas[i], #Theta tendra (125,785) la primera vez 
                    np.vstack((#Este Vstack tiene (785, 60000)
                        np.ones(len(X[0])).reshape(1,len(X[0])),
                        a[i]
                    )) 
                )#La mutiplicacion deja (125,60000)
            )#LA sigmoide lo deja de (125,60000)
        )
    return a

In [4]:
def cost_function(flat_thetas, theta_shapes, X, Y):
    _,m = Y.shape
    #print(m)
    a = feed_forward(
        inflate_matrixes(flat_thetas, theta_shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1-Y) * np.log(1 - a[-1])).sum() / m 

In [5]:
def backProp(flat_thetas, shapes, X, Y):
    _,m = Y.shape
    layers = len(shapes)+1
    thetas = inflate_matrixes(flat_thetas, shapes)
    
    #Paso 2.2
    a = feed_forward(thetas, X)
    
    #Paso 2.3
    deltas = [*range(layers-1), a[-1] - Y]
    
    #Paso 2.4
    for i in range(layers-2,0,-1):
        _, sinBias = thetas[i].shape
        deltas[i] = (np.matmul((np.delete(thetas[1], sinBias-1,1)).T,deltas[i+1])) * ((a[i])*(1-a[i]))
    
    #Paso 2.5
    for i in range(layers-1):
        deltaT[i] = deltaT[i] + np.matmul(deltas[i+1] ,
                                    (np.vstack(( np.ones(len(X[0])).reshape(1,len(X[0])), a[i]))).T)
    
    #Paso 3
    return flatten_list_of_arrays([
        np.true_divide(deltaT[i],m)
        for i in range(len(deltaT))
    ])

In [6]:
sigmoid = lambda x: 1.0/(1.0 + np.exp(-x))

In [7]:
flatten_list_of_arrays = lambda arrays: functools.reduce(
    lambda acc, v: np.array(
        [*acc.flatten(),*v.flatten()]
    ),arrays)

# HACER LA PRUEBA CON SOLO 5 DATOS DE ENTRENAMIENTO
# CON 10 DE SALIDA
# CON 8 EN MEDIO 

# Hora de Juntar todo Xd

In [8]:
X, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
X = X/1000.0

m = len(y_train)
y_train = y_train.reshape(m,1)

Y = (y_train == np.array(range(10))).astype(int)
Y = Y.T
X = X.T
a = [X]

In [9]:
NEURAL_NET = np.array([
    784,
    125,
    10
])
theta_shapes = np.hstack((
    NEURAL_NET[1:].reshape(len(NEURAL_NET) - 1, 1),
    (NEURAL_NET[:-1] + 1).reshape(len(NEURAL_NET) - 1, 1)
))
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

In [10]:
flat_thetas

array([0.57019204, 0.08165167, 0.50723208, ..., 0.88004148, 0.32826744,
       0.41323334])

In [15]:
inflar  = inflate_matrixes(flat_thetas, theta_shapes)
inflar[0]

array([[0.57019204, 0.08165167, 0.50723208, ..., 0.47527263, 0.30252466,
        0.08930025],
       [0.56980189, 0.53269995, 0.49737428, ..., 0.49723118, 0.49414868,
        0.33499227],
       [0.56817036, 0.95277738, 0.5474334 , ..., 0.36576188, 0.48642354,
        0.40713258],
       ...,
       [0.7600887 , 0.12221542, 0.77323518, ..., 0.00882607, 0.64862754,
        0.0279694 ],
       [0.90298039, 0.62646179, 0.72814112, ..., 0.68424434, 0.27976902,
        0.84343751],
       [0.09618244, 0.55203372, 0.44171892, ..., 0.90236414, 0.61919116,
        0.15688143]])

In [16]:
a = feed_forward(
        inflate_matrixes(flat_thetas, theta_shapes),
        X
    )

In [19]:
acero = a[0]
acero[:,0]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
       0.013, 0.073, 0.   , 0.   , 0.001, 0.004, 0.   , 0.   , 0.   ,
       0.   , 0.001, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.003, 0.   ,
       0.036, 0.136,

In [21]:
X[:,0]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
       0.013, 0.073, 0.   , 0.   , 0.001, 0.004, 0.   , 0.   , 0.   ,
       0.   , 0.001, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.003, 0.   ,
       0.036, 0.136,

In [105]:
pruebaCost = cost_function(flat_thetas, theta_shapes, X, Y)

60000


/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  


In [19]:
#theta0 = np.random.rand(125,785) * 0.001
#theta1 = np.random.rand(10,126) * 0.002
#thetas = [theta0, theta1]
#shapes = [thetas[0].shape, thetas[1].shape]

In [25]:
triangulito1 = np.zeros(theta_shapes[0], dtype = int)  
triangulito2 = np.zeros(theta_shapes[1], dtype = int)  

In [26]:
deltaT = [triangulito1, triangulito2] 

In [22]:
#flat_thetas = flatten_list_of_arrays(thetas)

In [100]:
costo = cost_function(flat_thetas, theta_shapes, X, Y)    

/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


In [87]:
back = backProp(flat_thetas, theta_shapes, X, Y)

In [27]:
result = minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args=(theta_shapes, X, Y),
    method='L-BFGS-B',
    jac= backProp,
    options={'disp': True, 'maxiter': 3000}
    
)

In [29]:
result.x

array([ 0.00031868,  0.0002991 ,  0.0001992 , ..., -0.04751215,
       -0.04743798, -0.04817217])